In [40]:
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda
from keras.models import Model
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
import os
from tqdm import *
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16 
from keras.applications.vgg16 import preprocess_input 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [31]:
def read_data_new(flag, path='../../../../assets/brand_images'):
    """
    read the dataset，dir format must be: ./train/, ./train.txt, ./test/, ./test.txt
    :param flag: 'train' or 'test'
    """
    content = open(os.path.join(path, '%s.txt' % flag))
    
    imgs = []
    labels = []

    lines = content.readlines()
    # tqdm is the progress bar, please install it with "pip install tqdm". if u wan't, you can replace tqdm(lines) with lines.
    for i in tqdm(lines):
        fname, y = i.replace('\n', '').split(' ')
        y = int(y)
        
        x = os.path.join(path, flag, fname)
        
        imgs.append(x)
        labels.append(y)
    return imgs, labels

# 掏粪

In [32]:
xx, yy = read_data_new('train')
# list to tensor
filelist_tensor = tf.convert_to_tensor(xx, dtype=tf.string)
labellist_tensor = tf.convert_to_tensor(yy, dtype=tf.int16)

100%|████████████████████████████████████████████████████████████████████████| 2725/2725 [00:00<00:00, 170355.31it/s]


In [4]:
partitions = [0] * len(yy)
testfilesize = int(len(yy) / 5)
partitions[:testfilesize] = [1] * testfilesize
random.shuffle(partitions)

# dynamic partition for split the train and test
trainfilelist, testfilelist = tf.dynamic_partition(data=filelist_tensor,
                                                  partitions=partitions,
                                                  num_partitions=2)

trainlabellist, testlabellist = tf.dynamic_partition(data=labellist_tensor,
                                                    partitions=partitions,
                                                    num_partitions=2)

In [5]:
# image width and height
IMG_WIDTH = 600
IMG_HEIGHT = 300

def image_operate(input_queue):
    """
    image operattions consist of file name to img, resize
    """
    label = input_queue[1]
    # get contents from file
    print(input_queue[0])
    contents = tf.read_file(input_queue[0])
    # decode image by its format
    image = tf.image.decode_jpeg(contents)
    # resize size argument must be [width, height]
    image = tf.image.resize_images(images=image,
                                   size=[IMG_WIDTH, IMG_HEIGHT])
    image = tf.reshape(image,tf.stack([IMG_WIDTH, IMG_HEIGHT, 3]))

    return image, label

In [6]:
# num_epochs 
train_input_queue = tf.train.slice_input_producer(tensor_list=[trainfilelist, trainlabellist],
                                                  shuffle=False,  # whether shuffle the data
                                                  num_epochs=1  # determines the number of times to output total data
                                                 )

In [19]:
train_x, train_y = image_operate(train_input_queue)

Tensor("input_producer/Gather:0", shape=(), dtype=string)


In [8]:
# batch size 
BATCH_SIZE = 20

train_batch = tf.train.batch(tensors=[train_x , train_y], 
                             batch_size=BATCH_SIZE,
                             num_threads=2,
                            )
train_batch

[<tf.Tensor 'batch:0' shape=(20, 600, 300, 3) dtype=float32>,
 <tf.Tensor 'batch:1' shape=(20,) dtype=int16>]

In [9]:
ep = 0
with tf.Session() as sess:
    ini_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(ini_op)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    try:  
        while not coord.should_stop():
            
            x, y_ = sess.run(train_batch)
            #  you can doing sth in there with the batch data x, y_
            ep += 1
            print('batch %s' % ep, y_)

#             print(2 / 0)
    except tf.errors.OutOfRangeError:
        print('Done training -- epoch limit reached')  
    finally:  
        coord.request_stop()
    coord.join(threads)

batch 1 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
batch 2 [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
batch 3 [2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
batch 4 [3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
batch 5 [4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
batch 6 [5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6]
batch 7 [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
batch 8 [6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
batch 9 [8 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
batch 10 [8 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
batch 11 [10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10]
batch 12 [10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11]
batch 13 [11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11]
batch 14 [11 11 11 11 11 11 12 12 12 12 12 12 12 12 12 12 12 12 12 12]
batch 15 [12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13]
batch 16 [13 13 13 13 13 13 13 13 13 13 13 13 14 14 14 14 14 14 14 14]
batch 17 [14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
ba

# Keras

In [41]:
datagen = ImageDataGenerator(
            rotation_range=0.2,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

In [55]:
global _gen_temp
_gen_temp = 0
sess = tf.Session()

def data_gen(batch_size=128, gene=4):
    global _gen_temp
    while True:
        label_list = yy[_gen_temp * batch_size: min((_gen_temp + 1) * batch_size, len(yy) - 1)]
        file_list = xx[_gen_temp * batch_size: min((_gen_temp + 1) * batch_size, len(xx) - 1)]
        _gen_temp += 1
        
        X_ = []
        for fname in file_list:
            img = cv2.imread(fname)
            img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
            X_.append(img.tolist())
        X_ = np.array(X_)
        y_ = np.array(label_list)
        
        i = 0
        X = None
        y = None
        
        for batch in datagen.flow(X_, y_, batch_size=batch_size):            
            if not type(X) == np.ndarray:
                X = batch[0]
                y = batch[1]
            else:
                X = np.concatenate([X, batch[0]], axis=0)
                y = np.concatenate([y, batch[1]], axis=0)
            
            i += 1
            if i >= gene:
                break
        
        yield np.array(X), np.array(y)

In [56]:
import time
tmp = time.time()
batchdata = next(data_gen(batch_size=128))
print(time.time() - tmp)

42.950504302978516
